In [1]:
pip install pandas sqlalchemy pymysql


Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os


In [2]:

load_dotenv()

def get_engine():
    engine = create_engine(
        f"mysql+pymysql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@"
        f"{os.getenv('DB_HOST')}/{os.getenv('DB_NAME')}"
    )
    return engine



In [15]:
def rentals_month(engine, month, year):
    query = f"""
    SELECT 
        customer_id,
        rental_date
    FROM rental
    WHERE MONTH(rental_date) = {month}
      AND YEAR(rental_date) = {year};
    """
    
    df = pd.read_sql(query, engine)
    return df


In [16]:
def rental_count_month(df, month, year):
    column_name = f"rentals_{month:02d}_{year}"
    
    result = (
        df.groupby("customer_id")
          .size()
          .reset_index(name=column_name)
    )
    
    return result


In [17]:
def compare_rentals(df1, df2):
    merged = pd.merge(
        df1,
        df2,
        on="customer_id",
        how="inner"
    )
    
    col1 = merged.columns[1]
    col2 = merged.columns[2]
    
    merged["difference"] = merged[col2] - merged[col1]
    
    return merged


In [18]:
# Retrieve rentals
may_rentals = rentals_month(engine, 5, 2005)
june_rentals = rentals_month(engine, 6, 2005)

# Count rentals per customer
may_counts = rental_count_month(may_rentals, 5, 2005)
june_counts = rental_count_month(june_rentals, 6, 2005)

# Compare months
comparison = compare_rentals(may_counts, june_counts)

comparison.head()


,customer_id,rentals_05_2005,rentals_06_2005,difference
0,1,2,7,5
1,2,1,1,0
2,3,2,4,2
3,5,3,5,2
4,6,3,4,1
